Combine CFB Game Data and College Names with NCES Postsecondary School Locations

In [105]:
import csv
from datetime import datetime
import requests
from time import sleep

import sys
sys.path.append('..')
from utils.cfb_api_utils import CFB_API_KEY

read_neutral_from_file = True

matched_schools_path = "../../intermediate_files/llm_matched_schools.csv"
school_locations_path = "../../postsecondary_school_locations/postsecondary_school_locations_2020-21_20251006_194802.csv"
intermediate_school_match_locs = "../../intermediate_files/schools_with_locations.csv"
game_data_path = "../../college_football_data/cfb_box-scores_2002-2024_20251006_193644.csv"
venue_data_path = "../../intermediate_files/collegefootballdata_venues_20251025_142553.csv"
neutral_games_not_found_path = "../../intermediate_files/neutral_games_not_found.csv"
neutral_games_found_path = "../../intermediate_files/neutral_games_found.csv"
output_path = "../../intermediate_files/cfb_with_locations.csv"

nces_to_coords = {}
school_file = open(school_locations_path, 'r')
reader = csv.DictReader(school_file)
for row in reader:
    nces_to_coords[row['NAME']] = {
        'lat': row['LAT'],
        'lon': row['LON']
    }
print(row)
print(list(nces_to_coords.keys())[0])
print(list(nces_to_coords.values())[0])

{'\ufeffX': '-122.49472', 'Y': '48.7911940000001', 'OBJECTID': '7012', 'UNITID': '496423', 'NAME': 'Zorganics Institute Beauty and Wellness', 'STREET': '410 WEST BAKERVIEW ROAD     SUITE 112', 'CITY': 'Bellingham', 'STATE': 'WA', 'ZIP': '98226', 'STFIP': '53', 'CNTY': '53073', 'NMCNTY': 'Whatcom County', 'LOCALE': '13', 'LAT': '48.791194', 'LON': '-122.49472', 'CBSA': '13380', 'NMCBSA': 'Bellingham, WA', 'CBSATYPE': '1', 'CSA': 'N', 'NMCSA': 'N', 'NECTA': 'N', 'NMNECTA': 'N', 'CD': '5302', 'SLDL': '53042', 'SLDU': '53042', 'SCHOOLYEAR': '2020-2021'}
Alabama A & M University
{'lat': '34.783368', 'lon': '-86.568502'}


In [106]:
combined_rows = []
match_file = open(matched_schools_path, 'r')
match_reader = csv.DictReader(match_file)
headers = match_reader.fieldnames + ['latitude', 'longitude']

for row in match_reader:
    nces_name = row['nces_name']
    coords = nces_to_coords.get(nces_name, {'lat': '', 'lon': ''})

    combined_row = row.copy()
    combined_row['latitude'] = coords['lat']
    combined_row['longitude'] = coords['lon']

    combined_rows.append(combined_row)

print(combined_rows[0])

{'cfb_school': 'Abilene Christian', 'nces_name': 'Abilene Christian University', 'processed_date': '2025-10-24T01:07:19.903300', 'latitude': '32.468943', 'longitude': '-99.709797'}


Save Intermediate School Match with Locations to File

In [107]:
output_file = open(intermediate_school_match_locs, 'w')
writer = csv.DictWriter(output_file, fieldnames=headers)
writer.writeheader()
writer.writerows(combined_rows)
output_file.close() 

Combine School Matches with Game Data

In [108]:
cfb_to_nces = {}
match_file = open(matched_schools_path, 'r')
match_reader = csv.DictReader(match_file)
for row in match_reader:
    cfb_to_nces[row['cfb_school']] = row['nces_name']

print(cfb_to_nces)

{'Abilene Christian': 'Abilene Christian University', 'Air Force': 'United States Air Force Academy', 'Akron': 'University of Akron Main Campus', 'Alabama': 'The University of Alabama', 'Alabama A&M': 'Alabama A & M University', 'Alabama State': 'Alabama State University', 'Albany': 'SUNY at Albany', 'Alcorn State': 'Alcorn State University', 'Appalachian State': 'Appalachian State University', 'Arizona': 'University of Arizona', 'Arizona State': 'Arizona State University Campus Immersion', 'Arkansas': 'University of Arkansas', 'Arkansas State': 'Arkansas State University', 'Arkansas-Pine Bluff': 'University of Arkansas at Pine Bluff', 'Army': 'United States Military Academy', 'Auburn': 'Auburn University', 'Austin Peay': 'Austin Peay State University', 'BYU': 'Brigham Young University-Provo', 'Ball State': 'Ball State University', 'Baylor': 'Baylor University', 'Bethune Cookman': 'Bethune-Cookman University', 'Boise State': 'Boise State University', 'Boston College': 'Boston College',

In [109]:
venue_id_to_loc = {}
match_file = open(venue_data_path, 'r')
match_reader = csv.DictReader(match_file)
for row in match_reader:
    venue_id_to_loc[int(row['id'])] = {
        'lat': row['latitude'],
        'lon': row['longitude']
    }

print(venue_id_to_loc)

{5938: {'lat': '34.1861922', 'lon': '-103.334397'}, 218: {'lat': '25.7525014', 'lon': '-80.3778912'}, 4779: {'lat': '25.055569', 'lon': '-77.358736'}, 11591: {'lat': '', 'lon': ''}, 5220: {'lat': '36.434053', 'lon': '-77.098431'}, 3884: {'lat': '', 'lon': ''}, 11589: {'lat': '', 'lon': ''}, 6043: {'lat': '40.816284', 'lon': '-73.589546'}, 11539: {'lat': '', 'lon': ''}, 11488: {'lat': '', 'lon': ''}, 5096: {'lat': '38.204444', 'lon': '-84.545556'}, 3926: {'lat': '34.286133', 'lon': '-81.622539'}, 5015: {'lat': '40.456862', 'lon': '-85.501228'}, 5083: {'lat': '40.456848', 'lon': '-85.501181'}, 5348: {'lat': '33.755578', 'lon': '-84.400881'}, 3689: {'lat': '32.37949', 'lon': '-86.293002'}, 538: {'lat': '33.8322043', 'lon': '-118.2517547'}, 4258: {'lat': '33.864', 'lon': '-118.261'}, 4205: {'lat': '32.7174202', 'lon': '-117.1627728'}, 3504: {'lat': '53.33523', 'lon': '-6.228447'}, 3603: {'lat': '36.0814337', 'lon': '-79.7700391'}, 3717: {'lat': '43.157285', 'lon': '-77.615214'}, 5810: {'la

In [110]:
def get_neutral_game_loc(home_team, away_team, year):
    if read_neutral_from_file:
        neutral_games_found_file = open(neutral_games_found_path, 'r')
        neutral_games_found = csv.DictReader(neutral_games_found_file)
        for row in neutral_games_found:
            if row['home'] == home_team and row['away'] == away_team and row['season'] == year:
                return {
                    'lat': row['game_lat'],
                    'lon': row['game_lon']
                }
        print("no neutral game loc found for ", home_team, away_team, year)
        return None


    url = "https://api.collegefootballdata.com/games" + "?year=" + year + "&home=" + home_team + "&away=" + away_team
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {CFB_API_KEY}"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    if not data:
        print("no data found for ", home_team, away_team, year)
        return None
    venue_id = data[0]['venueId']
    if not venue_id:
        print("no venue found for ", home_team, away_team, year)
        return None
    return venue_id_to_loc[venue_id]


In [111]:
cfb_game_data = open(game_data_path, 'r')
cfb_reader = csv.DictReader(cfb_game_data)

headers = cfb_reader.fieldnames + ['home_nces_name', 'home_lat', 'home_lon', 'away_nces_name', 'away_lat', 'away_lon', 'game_lat', 'game_lon', 'processed_time']

newrows = []

neutral_games_not_found = []

neutralCount = 0
for row in cfb_reader:
    row['home_nces_name'] = cfb_to_nces[row['home']]
    row['away_nces_name'] = cfb_to_nces[row['away']]
    home_nces_name = row['home_nces_name']
    away_nces_name = row['away_nces_name']
    row['home_lat'] = nces_to_coords[home_nces_name]['lat']
    row['home_lon'] = nces_to_coords[home_nces_name]['lon']
    row['away_lat'] = nces_to_coords[away_nces_name]['lat']
    row['away_lon'] = nces_to_coords[away_nces_name]['lon']
    row['processed_time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    if row['neutral'] == "False":
        row['game_lat'] = row['home_lat']
        row['game_lon'] = row['home_lon']
    else:
        print()
        neutralCount += 1
        neutral_game_loc = get_neutral_game_loc(row['home'], row['away'], row['season'])
        if not neutral_game_loc:
            print("skipping ", row['home'], row['away'], row['season'])
            neutral_games_not_found.append(row)
            continue
        print("found neutral game loc for ", row['home'], row['away'], row['season'])
        row['game_lat'] = neutral_game_loc['lat']
        row['game_lon'] = neutral_game_loc['lon']
        
        
    newrows.append(row)

print(newrows[0])
print("neutralCount=" , neutralCount)
print("neutral_games_not_found=", len(neutral_games_not_found))


no neutral game loc found for  Oklahoma Texas 2002
skipping  Oklahoma Texas 2002

no neutral game loc found for  Miami (FL) Ohio State 2002
skipping  Miami (FL) Ohio State 2002

no neutral game loc found for  Texas Oklahoma 2003
skipping  Texas Oklahoma 2003

found neutral game loc for  Oklahoma Texas 2004

no neutral game loc found for  Georgia Florida 2004
skipping  Georgia Florida 2004

found neutral game loc for  North Carolina Boston College 2004

no neutral game loc found for  Texas Oklahoma 2005
skipping  Texas Oklahoma 2005

no neutral game loc found for  Oklahoma Texas 2006
skipping  Oklahoma Texas 2006

no neutral game loc found for  Texas Oklahoma 2007
skipping  Texas Oklahoma 2007

no neutral game loc found for  East Carolina Virginia Tech 2008
skipping  East Carolina Virginia Tech 2008

found neutral game loc for  Clemson Alabama 2008

no neutral game loc found for  Missouri Illinois 2008
skipping  Missouri Illinois 2008

no neutral game loc found for  Colorado Colorado S

KeyError: 'Houston Christian University'

Write File

In [ ]:
cfb_with_locs =  open(output_path, 'w', newline='')
writer = csv.DictWriter(cfb_with_locs, fieldnames=headers)
writer.writeheader()
writer.writerows(newrows)

In [ ]:
neutral_games_not_found_file = open(neutral_games_not_found_path, 'w')
neutral_games_not_found_writer = csv.DictWriter(neutral_games_not_found_file, fieldnames=headers)
neutral_games_not_found_writer.writeheader()
neutral_games_not_found_writer.writerows(neutral_games_not_found)
neutral_games_not_found_file.close()

In [ ]:
neutral_games_found = [i for i in newrows if i not in neutral_games_not_found and i['neutral'] == "True"]

neutral_games_found_file = open(neutral_games_found_path, 'w')
neutral_games_found_writer = csv.DictWriter(neutral_games_found_file, fieldnames=headers)
neutral_games_found_writer.writeheader()
neutral_games_found_writer.writerows(neutral_games_found)
neutral_games_found_file.close()